In [3]:
import pandas as pd

# Load the pickle file back into a DataFrame
data = pd.read_pickle('data/simulated_data.pkl')

# Check the loaded data
print(data.head())

unique_id_count = data['id'].nunique()
print(f"Number of unique IDs: {unique_id_count}")

   id  visit  obstime  predtime  time  event         Y1        Y2        Y3  \
0   0      0        0         0     9   True  11.943728 -3.032593  2.760192   
1   0      1        1         1     9   True  12.255357 -5.431790  4.225383   
2   0      2        2         2     9   True  12.491947 -6.953460  2.854653   
3   0      3        3         3     9   True  16.406431 -8.508030  4.766191   
4   0      4        4         4     9   True  16.632347 -9.813989  5.816555   

    X1        X2    pred_Y1   pred_Y2   pred_Y3      true  
0  1.0  0.680195  11.943728 -3.032593  2.760192  1.000000  
1  1.0  0.680195  12.255357 -5.431790  4.225383  0.999397  
2  1.0  0.680195  12.491947 -6.953460  2.854653  0.998135  
3  1.0  0.680195  16.406431 -8.508030  4.766191  0.995494  
4  1.0  0.680195  16.632347 -9.813989  5.816555  0.989983  
Number of unique IDs: 1000


In [ ]:
import warnings 
warnings.filterwarnings("ignore")
n_epoch = 50
batch_size = 32
    
    
loss_values = []
loss1_list = []
loss2_list = []
for epoch in range(n_epoch):
    running_loss = 0
    train_id = np.random.permutation(train_id)
    for batch in range(0, len(train_id), batch_size):
        optimizer.zero_grad()
            
        indices = train_id[batch:batch+batch_size]
        batch_data = train_data[train_data["id"].isin(indices)]
            
        batch_long, batch_base, batch_mask, batch_e, batch_t, obs_time = get_tensors(batch_data.copy())
        batch_long_inp = batch_long[:,:-1,:].to('cuda')
        batch_long_out = batch_long[:,1:,:].to('cuda')
        batch_base = batch_base[:,:-1,:].to('cuda')
        batch_mask_inp = get_mask(batch_mask[:,:-1]).to('cuda')
        batch_mask_out = batch_mask[:,1:].unsqueeze(2).to('cuda') 
        obs_time = obs_time.to('cuda')
        yhat_long, yhat_surv = model(batch_long_inp, batch_base, batch_mask_inp,
                        obs_time[:,:-1].to('cuda'), obs_time[:,1:].to('cuda'))
        
        loss1 = long_loss(yhat_long, batch_long_out, batch_mask_out)
        loss2 = surv_loss(yhat_surv, batch_mask, batch_e)
        
        #loss = loss1 + loss2
        loss = multi_task_loss(loss1, loss2)
        
        loss.backward()
        scheduler.step()
        running_loss += loss
        loss1_list.append(loss1.tolist())
        loss2_list.append(loss2.tolist())
    loss_values.append(running_loss.tolist())
plt.plot((loss_values-np.min(loss_values))/(np.max(loss_values)-np.min(loss_values)), 'b-')
